In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

# Alternatives matrix and data
matrix = np.array([
    [0.181, 0.074, 0.067, 0.101],
    [0.078, 0.055, 0.049, 0.093],
    [0.021, 0.069, 0.035, 0.09],
    [0.033, 0.078, 0.076, 0.08],
    [0.028, 0.081, 0.235, 0.071],
    [0.304, 0.052, 0.112, 0.062],
    [0.038, 0.143, 0.032, 0.131],
    [0.014, 0.095, 0.045, 0.134],
    [0.028, 0.144, 0.136, 0.095],
    [0.014, 0.076, 0.148, 0.081],
    [0.009, 0.132, 0.065, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# Normalize matrix
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

norm_matrix = normalize_matrix(matrix, criteria_types)
df_norm = pd.DataFrame(norm_matrix, columns=criteria, index=sites)

# Criterion-wise rankings using scipy
print("\nRanking of sites for each criterion:")
criterion_ranks = pd.DataFrame(index=sites)

for i, c in enumerate(criteria):
    col = norm_matrix[:, i]
    if criteria_types[i] == 'max':
        ranks = rankdata(-col, method='min')  # Descending rank
    else:
        ranks = rankdata(col, method='min')   # Ascending rank
    criterion_ranks[c] = ranks

print(criterion_ranks)

# AHP weighted score and overall ranking
overall_scores = np.dot(norm_matrix, weights)
overall_ranks = rankdata(-overall_scores, method='min')  # Higher score = better

df_result = pd.DataFrame({
    'AHP Score': overall_scores,
    'Rank': overall_ranks
}, index=sites).sort_values('Rank')

print("\nOverall AHP Scores and Ranking:")
print(df_result)



Ranking of sites for each criterion:
        C1  C2  C3  C4
Site1    2   8   6   3
Site2    3  10   8   5
Site3    8   9  10   6
Site4    5   6   5   8
Site5    6   5   1   9
Site6    1  11   4  11
Site7    4   2  11   2
Site8    9   4   9   1
Site9    6   1   3   4
Site10   9   7   2   7
Site11  11   3   7  10

Overall AHP Scores and Ranking:
        AHP Score  Rank
Site9    0.579231     1
Site5    0.577337     2
Site7    0.382005     3
Site10   0.365235     4
Site6    0.346594     5
Site11   0.333916     6
Site1    0.287566     7
Site8    0.247106     8
Site4    0.216610     9
Site2    0.119638    10
Site3    0.097875    11
